In [1]:
import pandas as pd 
import os 
import re 
import time
from datetime import datetime
import sqlite3
import plotly.graph_objs as go
DATABASE = "Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"

In [2]:
import pandas as pd 
import os 
import re 
import time
from datetime import datetime
import sqlite3
import plotly.graph_objs as go

DATABASE = "Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"
PRODUCT = "614C"

def HISTORY_PRICE(DB, PRODUCT) :
    with sqlite3.connect(DB) as connection :
        query = f"SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE PRODUCT_CODE LIKE '%{PRODUCT}%'"
        df = pd.read_sql_query(query, connection)
    return df
PRODUCT_PRICE_HISTORY = HISTORY_PRICE(DATABASE, PRODUCT)
PRODUCT_PRICE_HISTORY = PRODUCT_PRICE_HISTORY.sort_values(by="QUOTE_DATE", inplace=True) 

In [3]:
def FORMATTED_TITLE(title): # split the name of the product if the length is more than 70
    # Find the index of the closest space to the 70th character
    split_index = title[:70].rfind(' ')
    if split_index == -1:
        split_index = 70
    # Split the string at the closest space index
    first_line = title[:split_index]
    second_line = title[split_index:].strip()  # Remove leading/trailing spaces if any

    # Combine the lines with a line break <br>
    formatted_title = first_line + "<br>" + second_line
    return formatted_title

In [4]:
def DRAW_PRICE_CHART(INPUT_DF) :
    try:
        table = INPUT_DF.tail(6)
    except:
        table = INPUT_DF
    num_rows = min(len(table), 6)
    fig = go.Figure()
    # Add a line trace to the figure with markers and vivid colors
    fig.add_trace(go.Scatter(x = table['QUOTE_DATE'], y = table['TOTAL_PRICE_M'], mode = 'lines+markers', 
                             marker=dict(size = 8, color = 'rgba(4, 25, 74, 1)'), 
                             line=dict(color = 'rgba(4, 25, 74, 1)'),
                             name='Price'))

    # Update the layout of the figure
    fig.update_layout(title=dict(text = FORMATTED_TITLE(table["DESCRIPTION"].iloc[-1])),
                  title_font=dict(size=15),  # Adjust title size
                  xaxis_title='Date',
                  yaxis_title='Price',
                  hovermode="x",
                  plot_bgcolor='rgba(135, 206, 250, 0.3)',
                  hoverlabel_bordercolor="rgba(135, 206, 250, 0.7)")

  # Change background color
    fig.update_traces(
        hovertemplate="<br>".join([
            "報價日期: %{x}",
            "價格: %{y}",
            "線材: %{customdata[0]}",  # Accessing column "WIRE_PRICE" from customdata
            "利潤: %{customdata[1]}",  # Accessing column "PROFIT_GROSS" from customdata
            "匯率: %{customdata[2]}",  # Accessing column "EXCHANGE_RATE" from customdata
        ]),
        customdata=[table.iloc[i,[13, 46, 48]] for i in range(num_rows)]  # Providing customdata for each trace
        )
    return fig

In [12]:
def DRAW_PROFIT_CHART(INPUT_DF) :
    try:
        table = INPUT_DF.tail(6)
    except:
        table = INPUT_DF
    num_rows = min(len(table), 6)
    fig = go.Figure()
    # Add a line trace to the figure with markers and vivid colors
    fig.add_trace(go.Scatter(x = table['QUOTE_DATE'], y = table['TOTAL_PRICE_M'], mode = 'lines+markers', 
                             marker=dict(size = 8, color = 'rgba(4, 25, 74, 1)'), 
                             line=dict(color = 'rgba(4, 25, 74, 1)'),
                             name='Price'))

    # Update the layout of the figure
    fig.update_layout(title=dict(text = FORMATTED_TITLE(table["DESCRIPTION"].iloc[-1])),
                  title_font=dict(size=15),  # Adjust title size
                  xaxis_title='Date',
                  yaxis_title='PRICE',
                  hovermode="x",
                  plot_bgcolor='rgba(135, 206, 250, 0.3)',
                  hoverlabel_bordercolor="rgba(135, 206, 250, 0.7)")

  # Change background color
    fig.update_traces(
        hovertemplate="<br>".join([
            "報價日期: %{x}",
            "價格: %{y}",
            "線材: %{customdata[0]}",  # Accessing column "WIRE_PRICE" from customdata
            "利潤: %{customdata[1]}",  # Accessing column "PROFIT_GROSS" from customdata
            "匯率: %{customdata[2]}",  # Accessing column "EXCHANGE_RATE" from customdata
        ]),
        customdata=[table.iloc[i,[13, 46, 48]] for i in range(num_rows)]  # Providing customdata for each trace
        )
    return fig

def HISTORY_PROFIT(DB, PRODUCT) :
    with sqlite3.connect(DB) as connection :
        query = f"SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE PRODUCT_CODE = '{PRODUCT}'"
        df = pd.read_sql_query(query, connection)
    return df
PRODUCT_PROFIT_HISTORY = HISTORY_PROFIT(DATABASE, "89151.035.0039.035").sort_values(by="QUOTE_DATE") 
# Show the chart
DRAW_PROFIT_CHART(PRODUCT_PROFIT_HISTORY).show()

In [6]:
def DRAW_PROFIT_CHART(INPUT_DF) :
    fig = go.Figure()
    # Add a line trace to the figure with markers and vivid colors
    for code in INPUT_DF["PRODUCT_CODE"].unique():
        INPUT_DF['QUOTE_DATE'] = pd.to_datetime(INPUT_DF['QUOTE_DATE'])
        table = INPUT_DF.loc[INPUT_DF["PRODUCT_CODE"] == code].sort_values(by = "QUOTE_DATE")
        fig.add_trace(go.Scatter(x = table['QUOTE_DATE'], 
                                 y = table['PROFIT_RATE'], 
                                 mode = 'lines+markers', 
                                 marker = dict(size = 8), 
                                 name = code ))
        refresh_variable = table[['WIRE_PRICE', 'PROFIT_RATE', 'EXCHANGE_RATE']].values
        fig.update_traces(
        # Providing customdata for each trace
        customdata = refresh_variable,
        hovertemplate="<br>".join([
            "報價日期: %{x}",
            "價格: %{y}",
            "線材: %{customdata[0]}",  # Accessing column "WIRE_PRICE" from customdata
            "利潤: %{customdata[1]}",  # Accessing column "PROFIT_GROSS" from customdata
            "匯率: %{customdata[2]}",  # Accessing column "EXCHANGE_RATE" from customdata
        ]))
    # Update the layout of the figure
        fig.update_layout(title=dict(text = FORMATTED_TITLE(table["DESCRIPTION"].iloc[-1])),
                  title_font=dict(size=15),  # Adjust title size
                  xaxis_title='Date',
                  yaxis_title='Profit rate',
                  hovermode="x",
                  plot_bgcolor='rgba(135, 206, 250, 0.3)',
                  hoverlabel_bordercolor="rgba(135, 206, 250, 0.7)",
                  yaxis2 = dict(
                      title = 'Product Code',
                      side = 'right',
                      overlaying = 'y',
                      tickvals = list(range(len(table["PRODUCT_CODE"].unique()))),
                      ticktext = table["PRODUCT_CODE"].unique().astype(str).tolist(),
                    ))
    return fig

def HISTORY_PROFIT(DB, matching) :
    with sqlite3.connect(DB) as connection :
        placeholders = ', '.join(['?'] * len(matching))
        query = f"SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE PRODUCT_CODE IN ({placeholders})"
        df = pd.read_sql_query(query, connection, params = matching)
    return df
mm = ["WSV2TR", "WSV3TR"]
a = HISTORY_PROFIT(DATABASE, mm)
DRAW_PROFIT_CHART(a)

In [65]:
b = a[['WIRE_PRICE', 'PROFIT_RATE', 'EXCHANGE_RATE']].values
b[1]

array([26.5 ,  0.06, 29.  ])

In [12]:
import pandas as pd
import plotly.graph_objects as go
import sqlite3

def DRAW_PROFIT_CHART(INPUT_DF):
    fig = go.Figure()

    # Convert QUOTE_DATE to datetime
    INPUT_DF['QUOTE_DATE'] = pd.to_datetime(INPUT_DF['QUOTE_DATE'])

    # Add a line trace for each product code
    for code in INPUT_DF["PRODUCT_CODE"].unique():
        table = INPUT_DF.loc[INPUT_DF["PRODUCT_CODE"] == code].sort_values(by="QUOTE_DATE")
        refresh_variable = table[['WIRE_PRICE', 'PROFIT_RATE', 'EXCHANGE_RATE']].values

        # Add trace for the current product code
        fig.add_trace(go.Scatter(
            x=table['QUOTE_DATE'],
            y=table['PROFIT_RATE'],
            mode='lines+markers',
            marker=dict(size=8),
            name=code,
            customdata=refresh_variable,  # Assign customdata for this trace
            hovertemplate="<br>".join([
                "報價日期: %{x}",
                "價格: %{y}",
                "線材: %{customdata[0]}",  # WIRE_PRICE
                "利潤: %{customdata[1]}",  # PROFIT_RATE
                "匯率: %{customdata[2]}",  # EXCHANGE_RATE
            ])
        ))

    # Update the layout of the figure (applied once, outside the loop)
    fig.update_layout(
        title=dict(text="Profit Rate Chart"),  # Replace FORMATTED_TITLE if undefined
        title_font=dict(size=15),
        xaxis_title='Date',
        yaxis_title='Profit rate',
        hovermode="x",
        plot_bgcolor='rgba(135, 206, 250, 0.3)',
        hoverlabel=dict(bordercolor="rgba(135, 206, 250, 0.7)"),
        yaxis2=dict(
            title='Product Code',
            side='right',
            overlaying='y',
            tickvals=list(range(len(INPUT_DF["PRODUCT_CODE"].unique()))),
            ticktext=INPUT_DF["PRODUCT_CODE"].unique().astype(str).tolist(),
        )
    )

    return fig
# Usage example
DATABASE = "your_database.db"
mm = ["614C", "620C"]
a = HISTORY_PROFIT(DATABASE, mm)

# Call the function to draw the chart
fig = DRAW_PROFIT_CHART(a)
fig.show()


DatabaseError: Execution failed on sql 'SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE PRODUCT_CODE IN (?, ?)': no such table: CUSTOMER_PRODUCT_SUMMARY